# index_points function
The index-points function is a central function within the overall model. At several stages within a single pass, the respective sampled points have to be selected and re-indexed for the next layer or function to process them. This notebook aims at specifically looking at this function with its respective inputs, as for the multi-scale-grouping (MSG) version of this model, currently we are receiving the following error: <br>
<br>
*RuntimeError: CUDA error: device-side assert triggered* <br>
<br>
This usually means, that there is something going sideways with the indices, I have found out. Therefore we will now adjust the function and test it in all its glory to finally get rid of this motherf****** error.

## Imports and Data

In [1]:
# IMPORTS
import torch
import numpy as np

## Previous definition of index_points
In order to evaluate the situation properly, I have inserted the commented and traditional verison of the function definition below. For clarity it has been renamed **index_points_old**.

In [ ]:
# Traditional definition of inddex_points
def index_points_old(points, idx):
    """
    Indexing points according to new group index.
    
    Input:
        points: input points data, [Batch/ Blocks, Num Points, Num Features]
        idx: sample index data, [Batch/ Blocks, New Num Points]
    Return:
        new_points: indexed points data, [Batches/ Blocks, New Num Points, Num Features]
    """
    device = points.device
    B = points.shape[0] # B is number of batches/ blocks
    # Defining views/ shapes for the re-indexing
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    # Batch/ Block indexing for input points according to previously defined views/ shapes
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    # Picking corresponding points from input points
    new_points = points[batch_indices, idx, :]
    
    # Return reindexed points according to batch/ block index and New Num Points
    return new_points

## Testable Hypotheses


Now, since the error seems to arise spontaneously at almost random iterations (or better said, to me random iterations), here you can find some ideas for what has been going sideways:<br>
- batch/block number/index does not match
- indices and points are on different devices 
- more will follow...

## Insights from Debugging

Here I will shortly list the central insights from debugging the old version of the index_points function, as they might help when trying to design the new self-scripted version, or when adjusting the traditional formulation.

After careful testing and adjsuting I found out, that it was an index issue, but most likely connected to the problem, that with the respective setting the model did not find enough points around centroids with the given radius. After increasing the radius and giving the model the opportunity to find enough sampels around centroids and also samples within the input point cloud, the problem is not apparent anymore. So this is important to be kept in mind. Nevertheless, it would be sueful to re-write this function for now, so that at least the data feed-in is constant and we are actually overtraining on one sample.

Therefore, as the funcitonality was proper before I learned that it is not the main funcitonality of the function which is an issue, but one has to keep an eye on the amount of points, the block size adn the radius to make sure the model does find enough centroids and sampels around the centroids to not run into an indexing issue, which unfortunately showcases itself in the less specific error mentioned above. 

## Solutions

After taking a look at the testable hypotheses, and also trying to evaluate those accordingly, in the following you will find the proposed solutions and their results.

### 1. Writing own re-indexing function

As the logic of it should be rather straightforward, my first initial thought is to re-write the entire function according to my own understanding of its functionality, then subsequent debugging might also be easier. A first draft of a version can be found below:

In [1]:
def index_points(points, idx):
    """
    Indexing points according to new group index.
    
    Input:
        points: input points data, [Batch/ Blocks, Num Points, Num Features]
        idx: sample index data, [Batch/ Blocks, New Num Points]
    Return:
        new_points: indexed points data, [Batches/ Blocks, New Num Points, Num Features]
    """
    
    device = points.device
    B = points.shape[0] # B is number of batches/ blocks
    
    # Check for index shape
    if idx.shape[0] < B:
        print(f"Batch number lower for indeces. Should be {B}, but is {idx.shape[0]}")
    elif idx.shape[0] > B:
        print(f"Batch number higher for indeces. Should be {B}, but is {idx.shape[0]}")
        
    # Defining shapes in respective dimension to be indexed
    #! This does not work as you are iamgining!! Instead both pointers point to the same object, which is not what we want
    batch_shape = sample_shape = list(idx.shape)
    batch_shape[1:] = sample_shape[0] = 1
    
    # Creating batch indices by indexing according to the shapes
    batch_indices = torch.arange(B, dtyp=torch.long).to(device).view(batch_shape).repeat(sample_shape)
    
    # Selecting corresponding point from input points
    new_points = points[batch_indices, idx, :]
    
    # Return reindexed points according to batch/ block index
    return new_points

## Testing-Cells

Below here you can find a collection of some simple cells in which I might test some basic hypotheses, but mostly about playing round with some single effects before piecing them together. 

### 1. Slicing and selecting from a multidimensional tensor 

One thing that I found confusing in the old version was the selection of the points from the input points to that function:<br>
*points[batch_indices, idx, :]*<br>
The reason I find it confusing is that the batch_indices is a 2D tensor of the shape [batches x num_points_NEW]. Therefore, we will have a short investigation into the slection of points from another tensor in this way. 


#### 1.1 Slicing and selecting with simplified dimensions

Before starting to slice and dice with the actual dimensions, I think it is the easiest to walk through it all in a simplified setting, therefore we will reduce the amount of rows and columns significantly to check for the working of the operations we are undertaking. 

In [19]:
# Creation of simplified tensor denoting points and index
points = torch.arange(start=0, end=24, step=1, dtype=torch.long).reshape(2,4,3)
idx = torch.tensor([[1,3], [0,2]])
# Creation of simplified shapes for re-indexing
view_shape = [2, 1]
repeat_shape = [1, 2]
# Creation of batch indices
batch_indices = torch.arange(2, dtype=torch.long).view(view_shape).repeat(repeat_shape)

# Print check 
print("------ Print check for sizes and batch_indices creation ------")
print(batch_indices.shape)
print(batch_indices[0].shape)
print(batch_indices[0])
print(batch_indices[1])
print("-----------")

# Slicing/ Selecting points with batch_indices
new_points = points[batch_indices, idx, :]

# Print Check
print("----- Print check for selected points -----")
print("Sizes and Shapes:")
print(f"points shape: {points.shape}")
print(f"new_points shape: {new_points.shape}")
print("-----------")
print("Tensor values:")
print(f"points tensor: {points}")
print(f"new_points tensor: {new_points}")

------ Print check for sizes and batch_indices creation ------
torch.Size([2, 2])
torch.Size([2])
tensor([0, 0])
tensor([1, 1])
-----------
----- Print check for selected points -----
Sizes and Shapes:
points shape: torch.Size([2, 4, 3])
new_points shape: torch.Size([2, 2, 3])
-----------
Tensor values:
points tensor: tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17],
         [18, 19, 20],
         [21, 22, 23]]])
new_points tensor: tensor([[[ 3,  4,  5],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [18, 19, 20]]])


Here, the main insight is, that the slicing/ selecting of the tensor does work as it should and it is a valid and valuable way of using the slicing or selection of points fromt he other tensor. 

#### 1.2 Slicing and selecting in network setting

After now trying the slicing on a simplified version, we now want to simulate a real tensor of the actual (or at least a possible) size within the network, to check for its funcitonality. 

In [4]:
# Creation of an input tensor of the size of input points
points = torch.arange(start=0, end=98304, step=1, dtype=torch.long).reshape(8, 4096,3)
idx = torch.randint(low=0, high=4096, size=(8, 1024))
# Creation of shapes used for re-indexing
B = 8
view_shape = [8,1]
repeat_shape = [1,1024]
batch_indices = torch.arange(B, dtype=torch.long).view(view_shape).repeat(repeat_shape)
new_points = points[batch_indices, idx, :]

# Print check
print("------ Checking new points created ------")
print(f"new_points shape: {new_points.shape}")


------ Checking new points created ------
new_points shape: torch.Size([8, 1024, 3])


Bottom line, after the investigation found that the indexing error does not arise from the function but rather from the sampling of the input point cloud, that is happening in the network. Therefore, not the biggest changes to the actual functionality are going to be inserted, but rather some changes to finally standardize the selection of points within the actual block of the batch. Because of the random selection of points in the getitem method of the dataset class, the point selectiopn and data input to the network was not uniform during the training, so that's why the network most likely had some trouble learning and also why the shapes adn sizes differed and at some point eventually lead to the idnexing error described above.